In [1]:
import Gpib
import time
import matplotlib
import matplotlib.pyplot as plt

import ipywidgets as widgets

import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import numpy as np

%matplotlib inline


In [6]:
import smu as sm
import meas
inst = Gpib.Gpib(0, 2)
smu = sm.hp4142b(inst)


HEWLETT PACKARD,4142B,0,4.30



In [19]:

Ic_max=0.2

#2N3904 EBC pinout

S=0
G=4
D=2

channels=['X', 'D', 'Y', 'G']

def meas_pfet(Vgs=0.0, Vds_max=-10, Ids_max=0.1, Igs_max=-1e-3, P_max=1):    
    smu.connect([G,D])
    smu.voltage(G, Vgs, i_lim=Igs_max)
  
    smu.sweep_v(D, start=-0.01, stop=Vds_max, i_lim=Ids_max, mode='log', p_lim=P_max)
    smu.write("MM2,2,4\n")

    smu.xe()
    #Wait for 5 seconds, larger FET needs a lot of autoscaling in measurements due to 
    #curreny limits being hit all the time
    time.sleep(5)
    #print(smu.errors())

    smu.zero()
    #print(smu.opstat())

    time.sleep(0.5)
    smu.disconnect()

    r=smu.readresult()
    #print("RESULT: %s" % str(r))
    d=smu.parseresult(r, channels)

    return d

def meas_vgs_ids(Vds=-5, Vgs_min=0, Vgs_max=-12, Ids_max=-10e-3):
    smu.connect([G,D])
    smu.voltage(D, Vds, i_lim=Ids_max)
  
    smu.sweep_v(G, start=Vgs_min, stop=Vgs_max, i_lim=1e-3)
    smu.write("MM2,2,4\n")

    smu.xe()
    smu.zero()
    smu.disconnect()

    r=smu.readresult()
    #print("RESULT: %s" % str(r))
    d=smu.parseresult(r, channels)

    return d


def meas_idss(Vds=-10.0, Vgs=0.0, Ids_max=-1e-3):
    smu.connect([G,D])
    smu.voltage(G, Vgs, i_lim=1e-6)
    smu.voltage(D, Vds, i_lim=Ids_max)
    
    smu.write("MM1,2,4")
    smu.xe()
    smu.disconnect()
    r=smu.readresult()
    d=smu.parseresult(r, channels)
    return d['I_D'][0]

def meas_vgs_on(Ids=10e-3, Vds=-10, Ids_max=0.15):
    smu.connect([D,G])
    smu.search(G, start=-0.01, stop=-10, rate=10, i_lim=1e-3)
    smu.sense_i(D, v=Vds, i_target=-Ids, i_lim=2*Ids) 
    smu.search_timing(hold=10e-3, delay=10e-3)
    smu.search_cfg(op='FBpos', meas="searchVsenseVI") #ASM 1,4,5e-4
    #smu.search_cfg(op='ramp_gt', meas="searchVsenseVI", itime=1e-3) #ASM 1,4,5e-4
    
    smu.write("MM6")
    smu.xe()
    #time.sleep(2)
    #print(smu.opstat())
    #print(smu.errors())
    smu.zero()
    smu.disconnect()
    r=smu.readresult()
    #print(r)
    d=smu.parseresult(r, channels)
    return d['V_G'][0]

    

In [20]:
print(meas_idss(Vgs=-4, Vds=-0.5, Ids_max=1))
print(meas_vgs_on(Ids=1e-3))


IBSTA: Complete END
-0.072196
IBSTA: Complete END
-3.3952


In [ ]:
#Measure n JFET operating curves
data = []

results = []

#for vgs in np.arange(-3, -7, -1):
for vgs in [-4, -6, -8, -10]:

    d=meas_pfet(Vgs=float(vgs), Vds_max=-5, Ids_max=1, P_max=1)
    print(vgs)
    ids=np.array(d['I_D'])
    igs=np.array(d['I_G'])
    vds=np.array(d['V_D'])
    results.append ( { 'Ids': ids,
                        'Igs': igs,
                        'Vds': vds,
                        'Vgs': vgs,
                      'part': '2N7000',
                     't': time.time() } )

    
print("READY.")


IBSTA: Complete END
-4
IBSTA: Complete END
MEAS_ERR: CBI-0.99998E+00
MEAS_ERR: TDI-0.56238E-09
MEAS_ERR: VBI+199.999E+99
-6


In [10]:
data= []

for r in results:
    trace =  go.Scatter(x=r['Vds'], y=1e3*r['Ids'], name='Vgs=%.2f' % (r['Vgs']))
    data.append(trace)

layout = go.Layout(
    title='D8P05',
    xaxis=dict(
        title='V_ds(V)',
        autorange='reversed',

        #type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='I_ds(mA)',
        autorange='reversed',

        #range=[0, 300],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout })

## data= []

for r in results:
    trace =  go.Scatter(x=r['Vds'], y=1e3*r['Ids'], name='Vgs=%.2f' % (r['Vgs']))
    data.append(trace)

layout = go.Layout(
    title='2SK170',
    xaxis=dict(
        title='V_ds(mA)',
        #type='log',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    ),
    yaxis=dict(
        title='I_ds(mA)', 
        #range=[0, 300],
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#000000'
        )
    )
)


iplot({ 'data': data, "layout": layout })